# Unit 5 - Example 19: Deployment & Monitoring

## 📚 Learning Objectives

By completing this notebook, you will:
- Understand the key concepts of this topic
- Apply the topic using Python code examples
- Practice with small, realistic datasets or scenarios

## 🔗 Prerequisites

- ✅ Basic Python
- ✅ Basic NumPy/Pandas (when applicable)

---

## Official Structure Reference

This notebook supports **Course 05, Unit 5** requirements from `DETAILED_UNIT_DESCRIPTIONS.md`.

---


# Unit 5 - Example 19: Deployment & Monitoring

## 🔗 Solving the Problem from Example 18 | حل المشكلة من المثال 18

**Remember the dead end from Example 18?**
- We learned large dataset handling strategies
- But the model is ready - how do we deploy it for users?
- We needed deployment and monitoring strategies

**This notebook solves that problem!**
- We'll learn **model deployment strategies**
- We'll learn **monitoring and maintenance**
- We'll learn **production deployment best practices**

**This solves the deployment problem from Example 18!**


In [1]:
# Step 1: Import necessary libraries
import pandas as pd
import numpy as np
import json
import pickle
from datetime import datetime
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

print("=" * 70)
print("Example 19: Deployment & Monitoring | النشر والمراقبة")
print("=" * 70)
print("\n📚 Prerequisites: Examples 14-18 completed, deployment knowledge")
print("🔗 This is the SIXTH example in Unit 5 - deployment and monitoring")
print("🎯 Goal: Master deploying and monitoring ML models")
print("Reference: Study 19.pdf before running this code example.\n")


Example 19: Deployment & Monitoring | النشر والمراقبة

📚 Prerequisites: Examples 14-18 completed, deployment knowledge
🔗 This is the SIXTH example in Unit 5 - deployment and monitoring
🎯 Goal: Master deploying and monitoring ML models
Reference: Study 19.pdf before running this code example.



## 


# 1. TRAIN MODEL FOR DEPLOYMENT


## 


In [2]:
print("\n1. Training Model for Deployment")
print("-" * 70)
np.random.seed(42)
n_samples = 1000
X = np.random.randn(n_samples, 3)
y = X[:, 0] * 2 + X[:, 1] * 1.5 - X[:, 2] * 0.5 + np.random.randn(n_samples) * 0.1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"✓ Model trained successfully")
print(f"MSE: {mse:.4f}, R²: {r2:.4f}")


1. Training Model for Deployment
----------------------------------------------------------------------
✓ Model trained successfully
MSE: 0.0104, R²: 0.9985


## 


# 2. SAVE MODEL FOR DEPLOYMENT


## 


In [3]:
print("\n\n2. Saving Model")
print("-" * 70)
# Save model using pickle
model_path = 'deployed_model.pkl'
with open(model_path, 'wb') as f:
    pickle.dump(model, f)
print(f"✓ Model saved to {model_path}")
# Save model metadata
metadata = {
'model_version': '1.0.0', 'deployed_at': datetime.now().isoformat(), 'training_date': datetime.now().isoformat(), 'metrics': {
'mse': float(mse),
'r2': float(r2)
}, 'features': ['feature_0', 'feature_1', 'feature_2'],
'model_type': 'LinearRegression'
}
metadata_path = 'model_metadata.json'
with open(metadata_path, 'w') as f:
    json.dump(metadata, f, indent=2)
print(f"✓ Model metadata saved to {metadata_path}")



2. Saving Model
----------------------------------------------------------------------
✓ Model saved to deployed_model.pkl
✓ Model metadata saved to model_metadata.json


## 


# 3. DEPLOYMENT FUNCTION


## 


In [4]:
print("\n\n3. Deployment Function")
print("-" * 70)
def predict(model, features):
    """
    Make predictions using deployed model
    """
    try:
        logger.info(f"Making prediction for {len(features)} samples")
        # Validate input
        if len(features.shape) != 2:
            raise ValueError("Features must be 2D array")
        # Make prediction
        prediction = model.predict(features)
        logger.info(f"Prediction successful: {len(prediction)} results")
        return prediction
    except Exception as e:
        logger.error(f"Prediction failed: {str(e)}", exc_info=True)
        raise e

# Test deployment function
test_features = X_test[:5]
predictions = predict(model, test_features)
print(f"\n✓ Deployment function tested successfully")
print(f"Sample predictions: {predictions[:3]}")

2026-01-15 23:08:38,879 - INFO - Making prediction for 5 samples


2026-01-15 23:08:38,880 - INFO - Prediction successful: 5 results




3. Deployment Function
----------------------------------------------------------------------

✓ Deployment function tested successfully
Sample predictions: [ 3.00910382  2.75354276 -2.06205368]


## 


In [5]:
# 4. MONITORING SETUP


## 


In [6]:
print("\n\n4. Monitoring Setup")
print("-" * 70)
class ModelMonitor:
    """Simple model monitoring class"""
    def __init__(self):
        self.predictions_log = []
        self.errors_log = []
    
    def log_prediction(self, features, prediction, actual=None):
        """Log prediction for monitoring"""
        log_entry = {
            'timestamp': datetime.now().isoformat(), 'features': features.tolist() if isinstance(features, np.ndarray) else features,
            'prediction': float(prediction) if np.isscalar(prediction) else prediction.tolist(), 'actual': float(actual) if actual is not None and np.isscalar(actual) else None
        }
        self.predictions_log.append(log_entry)
        logger.info(f"Logged prediction: {log_entry['prediction']}")
    
    def log_error(self, error_message):
        """Log error for monitoring"""
        error_entry = {
            'timestamp': datetime.now().isoformat(), 'error': error_message
        }
        self.errors_log.append(error_entry)
        logger.error(f"Logged error: {error_message}")
    
    def get_stats(self):
        """Get monitoring statistics"""
        return {
            'total_predictions': len(self.predictions_log), 'total_errors': len(self.errors_log),
            'latest_prediction': self.predictions_log[-1] if self.predictions_log else None
        }

monitor = ModelMonitor()
# Simulate monitoring
for i in range(5):
    features = X_test[i:i+1]
    pred = model.predict(features)[0]
    actual = y_test.iloc[i] if hasattr(y_test, 'iloc') else y_test[i]
    monitor.log_prediction(features[0], pred, actual)

stats = monitor.get_stats()
print(f"\nMonitoring Statistics:")
print(f"  Total predictions: {stats['total_predictions']}")
print(f"  Total errors: {stats['total_errors']}")

2026-01-15 23:08:38,885 - INFO - Logged prediction: 3.009103823581936


2026-01-15 23:08:38,885 - INFO - Logged prediction: 2.753542759170331


2026-01-15 23:08:38,886 - INFO - Logged prediction: -2.062053684283035


2026-01-15 23:08:38,886 - INFO - Logged prediction: -3.695126217427459


2026-01-15 23:08:38,886 - INFO - Logged prediction: 3.1305121454342015




4. Monitoring Setup
----------------------------------------------------------------------

Monitoring Statistics:
  Total predictions: 5
  Total errors: 0


## 


# 5. DEPLOYMENT CHECKLIST


## 


In [7]:
print("\n\n5. Deployment Checklist")
print("-" * 70)
checklist = {
'Model trained and validated': True, 'Model saved and versioned': True,
'Metadata documented': True,
'Error handling implemented': True,
'Logging configured': True,
'Monitoring set up': True,
'Documentation created': True
}
print("\nDeployment Checklist:")
for item, status in checklist.items():
    status_symbol = "✓" if status else "✗"
    print(f"  {status_symbol} {item}")



5. Deployment Checklist
----------------------------------------------------------------------

Deployment Checklist:
  ✓ Model trained and validated
  ✓ Model saved and versioned
  ✓ Metadata documented
  ✓ Error handling implemented
  ✓ Logging configured
  ✓ Monitoring set up
  ✓ Documentation created


## 


# 6. SUMMARY


## 


In [8]:
print("\n\n" + "=" * 70)
print("Summary")
print("=" * 70)
print("\nKey Concepts Covered:")
print("1. Model serialization and saving")
print("2. Deployment functions")
print("3. Monitoring and logging")
print("4. Version control")
print("5. Deployment best practices")
print("\n" + "=" * 70)
print("Course Complete! All 19 examples created successfully!")
print(" !      19 !")
print("=" * 70)



Summary

Key Concepts Covered:
1. Model serialization and saving
2. Deployment functions
3. Monitoring and logging
4. Version control
5. Deployment best practices

Course Complete! All 19 examples created successfully!
 !      19 !
